# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import *
from tqdm.auto import tqdm # pip install tqdm

Here we use $H_2$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [2]:
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(h2)
print('Number of fragments: {}\n'.format(len(qwc_list)))
for i, fragment in enumerate(qwc_list):
    print(f'Fragments {i} \n{fragment}\n')

Number of fragments: 5

Fragments 0 
-0.32760818967480915 [] +
0.049197645871367574 [X0 Y1 Y2 X3]

Fragments 1 
-0.049197645871367574 [X0 X1 Y2 Y3]

Fragments 2 
-0.049197645871367574 [Y0 Y1 X2 X3]

Fragments 3 
0.049197645871367574 [Y0 X1 X2 Y3]

Fragments 4 
0.13716572937099503 [Z0] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856082 [Z0 Z2] +
0.15542669077992838 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992838 [Z1 Z2] +
0.10622904490856082 [Z1 Z3] +
-0.13036292057109122 [Z2] +
0.1632676867356435 [Z2 Z3] +
-0.13036292057109122 [Z3]



By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_2$ by partitioning its terms into mutually commuting fragments. 

In [3]:
comm_groups = get_commuting_group(h2)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('\n')
print('The first commuting group')
print(comm_groups[1])
print('\n')
print('The second commuting group')
print(comm_groups[2])

Number of mutually commuting fragments: 2


The first commuting group
-0.32760818967480915 [] +
-0.049197645871367574 [X0 X1 Y2 Y3] +
0.049197645871367574 [X0 Y1 Y2 X3] +
0.049197645871367574 [Y0 X1 X2 Y3] +
-0.049197645871367574 [Y0 Y1 X2 X3] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856082 [Z0 Z2] +
0.15542669077992838 [Z0 Z3] +
0.15542669077992838 [Z1 Z2] +
0.10622904490856082 [Z1 Z3] +
0.1632676867356435 [Z2 Z3]


The second commuting group
0.13716572937099503 [Z0] +
0.13716572937099503 [Z1] +
-0.13036292057109122 [Z2] +
-0.13036292057109122 [Z3]


To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [4]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999996+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [6]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-0.3276081896748089 [] +
0.15542669077992824 [X0] +
0.15660062488237952 [X0 X1] +
0.049197645871367546 [X0 X1 Z3] +
0.10622904490856076 [X0 X2] +
-0.049197645871367546 [X0 Z3] +
0.10622904490856076 [X1] +
0.15542669077992824 [X1 X2] +
-0.049197645871367546 [X1 X2 Z3] +
0.16326768673564335 [X2] +
0.049197645871367546 [X2 Z3]


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [7]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-0.3276081896748088 [] +
0.15542669077992818 [Z0] +
0.15660062488237944 [Z0 Z1] +
0.04919764587136753 [Z0 Z1 Z3] +
0.1062290449085607 [Z0 Z2] +
-0.04919764587136753 [Z0 Z3] +
0.1062290449085607 [Z1] +
0.15542669077992818 [Z1 Z2] +
-0.04919764587136753 [Z1 Z2 Z3] +
0.16326768673564326 [Z2] +
0.04919764587136753 [Z2 Z3]


## Class for measurement grouping

In [16]:
class MeasurementGrouper():
    """
    Usage:
        # run the grouping algorithm
        h2o_mcalc = MeasurementGrouper('h2o')
        # get commuting groups
        comm_groups = h2o_mcalc.comm_groups
        # get qwc unitaries
        uqwcs = h2o_mcalc.uqwcs
        # get qwc groups
        qwc_groups = h2o_mcalc.qwc_groups
        # get z unitaries
        uqwcs = h2o_mcalc.uzs
        # get z groups
        z_groups = h2o_mcalc.z_groups
    """
    
    def __init__(self, mol_sym, basis='sto3g', mapping='jw', do_taper=True, apply_uzs=False, verbose=True):
        """
        parameters:
            mol_sym: string representing the molecule type

            basis: orbital basis
            mapping: fermion -> qubit mapping. 'jw' (jordan-wigner) is defualt
            do_taper: whether to do tapering after the mapping
            simulate_gates: whether to apply last set of unitaries for transforming
                qwc groups to z-basis. This is not actually needed if all you want
                is the unitaries, and is only for informational purposes. It takes
                a while to run on n2 and nh3
            verbose: whether to print steps
        """
        
        self.verbose = verbose
        
        # just a quick check to make sure the molecule is implemented
        valid_mol_syms = ['h2', 'h2o', 'lih', 'n2', 'nh3', 'h4']
        assert mol_sym in valid_mol_syms, \
            f"Provided invalid mol_sym: '{mol_sym}'. Please provide a mol_sym in {valid_mol_syms}"
        
        # lookup tables
        # choosing the minima from task 1 but not sure this is the right thing to do
        lut_geos = {
            'h2': 0.7,
            'h2o': 1,
            'lih': 1.5,
            'n2': 1.2,
            'nh3': 1.1,
            'h4': 95 # not the minimum, chose an angle
        }
        # literally counting the number of electrons from periodic table
        # but not sure this is the right way to do it
        lut_n_electrons = {
            'h2': 2,
            'h2o': 10,
            'lih': 4,
            'n2': 14,
            'nh3': 10,
            'h4': 4
        }
    
        # NOW FOLOW THE STEPS
        self._print("1) Initialise the molecule object in a given basis and with a provided mapping")
        self.mol = get_qubit_hamiltonian(
            mol=mol_sym, geometry=lut_geos[mol_sym], basis=basis, qubit_transf=mapping)
        n_orbitals = self._get_num_orbitals(self.mol)
        self._print(f"\t{n_orbitals} orbitals")
        
        self._print("2) Optionally apply tapering")
        if do_taper:
            self.mol = taper_hamiltonian(self.mol, n_spin_orbitals=self._get_num_orbitals(self.mol),
                                        n_electrons=lut_n_electrons[mol_sym], qubit_transf=mapping)
            n_orbitals = self._get_num_orbitals(self.mol)
            self._print(f"\t{n_orbitals} orbitals")
        
        if n_orbitals > 8:
            self._print("More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups")
            qwc_groups = get_qwc_group(self.mol)
            # switch structure for consistency with comm_groups structure
            self.qwc_groups = {i+1: v for i, v in enumerate(qwc_groups)}
            self._print(f"\t{len(self.qwc_groups)} QWC groups," \
                + f" a {len(self.mol.terms)/ len(self.qwc_groups):.2f} x reduction from {len(self.mol.terms)} terms")
        else:
            self._print("3) Get commuting groups")
            self.comm_groups = get_commuting_group(self.mol)
            self._print(
                f"\t{len(self.comm_groups)} commuting groups," \
                + f" a {len(self.mol.terms)/ len(self.comm_groups):.2f} x reduction from {len(self.mol.terms)} terms")

            self._print("4) For each commuting group get the unitary for transforming to qwc")
            self.uqwcs = {}
            for group_ix in tqdm(self.comm_groups, disable=(not self.verbose)): # comm groups is a dict with int keys
                self.uqwcs[group_ix] = get_qwc_unitary(self.comm_groups[group_ix])
        
            self._print("5) Apply unitaries to get the qwcs")
            self.qwc_groups = {}
            for group_ix in tqdm(self.comm_groups, disable=(not self.verbose)):
                self.qwc_groups[group_ix] = remove_complex(
                    self.uqwcs[group_ix] * self.comm_groups[group_ix] * self.uqwcs[group_ix])
            
        self._print("6) Get unitaries for rotating everything to the measurable z-basis")
        self.uzs = {}
        for group_ix in tqdm(self.qwc_groups, disable=(not self.verbose)):
            self.uzs[group_ix] = get_zform_unitary(self.qwc_groups[group_ix])
        
        self.z_groups = {}
        if apply_uzs:
            self._print("7) Finally, apply latter unitaries to move all qwc groups to z-basis")
            for group_ix in tqdm(self.qwc_groups, disable=(not self.verbose)):
                self.z_groups[group_ix] = remove_complex(
                    self.uzs[group_ix] * self.qwc_groups[group_ix] * self.uzs[group_ix])
        
    def _print(self, string):
        if self.verbose:
            print(string)
    
    def _get_num_orbitals(self, mol):
        """
        Hacky way to get number of orbitals for a molecule object
        Probably there is a better way to do it
        """
        max_orbital = 0
        for product_op in mol.terms:
            if product_op:
                mx = max([op[0] for op in product_op])
                if mx > max_orbital:
                    max_orbital = mx
        return max_orbital + 1

# H<sub>2</sub> (again - with the new framework)

In [22]:
mcalc = MeasurementGrouper('h2')
print('\n')
print('First commuting group')
print(mcalc.comm_groups[1])
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First qwc unitary')
print(mcalc.uqwcs[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	4 orbitals
2) Optionally apply tapering
	1 orbitals
3) Get commuting groups
	2 commuting groups, a 1.50 x reduction from 3 terms
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First commuting group
-0.2764376754246701 [] +
0.17900057606140662 [X0]


First QWC group
-0.27643767542467007 [] +
0.1790005760614066 [Z0]


First Z group
-0.27643767542467007 [] +
0.1790005760614066 [Z0]


First qwc unitary
0.7071067811865475 [X0] +
0.7071067811865475 [Z0]


First z unitary
1.0 []


## H<sub>2</sub>O
Skip getting commuting groups as these steps are intractable. Skip straight ahead to getting QWC groups.

In [17]:
mcalc = MeasurementGrouper('h2o', apply_uzs=True)
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	14 orbitals
2) Optionally apply tapering
	10 orbitals
More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups
	200 QWC groups, a 5.17 x reduction from 1035 terms
6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First QWC group
-46.35740160289526 [] +
0.0021618990497094402 [Y0 Z1 Z2 Z3 Z5 Z6 Y8] +
-0.0021618990497094402 [Y0 Z1 Z2 Y8] +
-0.0017952267657413302 [Y0 Z1 Z3 Z4 Z6 X7 Y8 X9] +
-0.004721276878475687 [Y0 Y8 X9] +
1.6462328281412955 [Z1] +
0.12333444961877364 [Z1 Z2] +
0.0037537905436206037 [Z1 Z2 Z3 Z5 Z6 X9] +
0.16866356664010346 [Z1 Z3] +
0.13693480693261512 [Z1 Z4] +
0.3378644911679983 [Z1 Z5] +
0.15105317022508946 [Z1 Z6] +
1.185904752943987 [Z2] +
0.14716615820292295 [Z2 Z3] +
0.002210090959302922 [Z2 Z3 Z4 Z6 X7 X9] +
0.13496957191310968 [Z2 Z4] +
0.30264028366319506 [Z2 Z5] +
0.1408322772493606 [Z2 Z6] +
1.294980147036568 [Z3] +
0.191189227748051 [Z3 Z4] +
0.3472114393571394 [Z3 Z5] +
0.11777797258841 [Z3 Z6] +
1.294980147036568 [Z4] +
0.3472114393571394 [Z4 Z5] +
0.13636442553512224 [Z4 Z6] +
2.730246966069517 [Z5] +
0.2819519797578531 [Z5 Z6] +
0.7861029297052462 [Z6]


First Z group
-46.35740160289523 [] +
0.002161899049709439 [Z0 Z1 Z2 Z3 Z5 Z6 Z8] +
-0.002161899049709439 

## H<sub>4</sub>

In [24]:
mcalc = MeasurementGrouper('h4', apply_uzs=True)
print('\n')
print('First commuting group')
print(mcalc.comm_groups[1])
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First qwc unitary')
print(mcalc.uqwcs[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	8 orbitals
2) Optionally apply tapering
	4 orbitals
3) Get commuting groups
	5 commuting groups, a 9.20 x reduction from 46 terms
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First commuting group
-0.6362710050108025 [] +
-0.07052385292238096 [X0 Z1 Z2] +
0.07052385292238096 [X0 Z2 Z3] +
0.028555850825589 [Y0 X1 Y3] +
-0.005103767726276638 [Y0 Y1 X3] +
0.07114732950498256 [Y0 Z1 Y2 Z3] +
-0.07114732950498256 [Y0 Y2] +
0.07114732950498256 [Z0 X1 Z2 X3] +
0.07114732950498256 [Z0 Y1 Z2 Y3] +
0.005103767726276638 [Z0 Z1 X2] +
-0.028555850825589 [Z0 X2 Z3] +
-0.07811611442288481 [X1 Y2 Y3] +
0.07811611442288481 [Y1 Y2 X3] +
0.32806903436877016 [Z1 Z3]


First QWC group
-0.636271005010802 [] +
0.07052385292238092 [Z0] +
-0.07052385292238092 [Z0 X1] +
0.0711473295049825 [Z0 X1 Z2] +
-0.005103767726276633 [Z0 X1 Z3] +
-0.0711473295049825 [Z0 Z2] +
0.02855585082558898 [Z0 Z3] +
0.32806903436876994 [X1] +
0.005103767726276633 [X1 Z2] +
0.07811611442288477 [X1 Z2 Z3] +
-0.0711473295049825 [X1 Z3] +
-0.02855585082558898 [Z2] +
-0.07811611442288477 [Z2 Z3] +
0.0711473295049825 [Z3]


First Z group
-0.6362710050108019 [] +
0.0705238529223809 [Z0] +
-0.0705238529223809

## LiH

In [8]:
mcalc = MeasurementGrouper('lih', apply_uzs=True)
print('\n')
print('First commuting group')
print(mcalc.comm_groups[1])
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First qwc unitary')
print(mcalc.uqwcs[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	12 orbitals
2) Optionally apply tapering
	8 orbitals
3) Get commuting groups
	41 commuting groups
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First commuting group
-3.947119127137718 [] +
-0.0009148940971918733 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-5.845579096240596e-05 [X0 X2 X3] +
0.0009148940971918733 [Y0 Y1 X2 Z3 Z4 Z5 Z7] +
-5.845579096240596e-05 [Y0 Y2 X3] +
0.011733623912046789 [Z0 X1 Z2 X3] +
-0.0028574469714684443 [Z0 Y1 Y3 Z4 Z5 Z6] +
-0.00010502189355807878 [Z0 Y1 Z3 Z4 Z5 Y6 X7] +
0.09486647602250253 [Z0 Z1 Z3 Z4 Z5 Z7] +
0.05362141072261479 [Z0 Z2] +
0.0010456709574065426 [Z0 Z2 X3 Z4 Z5 Y6 Y7] +
0.08860943826798089 [Z0 Z2 Z4 Z5] +
0.09041974869932769 [Z0 Z2 Z4 Z5 Z6 Z7] +
-0.0015406700896984202 [X1 X3] +
-2.844458479947759e-05 [Y1 Z2 Y3 Z4 Z5 Z7] +
0.010540187409003523 [Y1 Z2 Y3 Z6] +
0.0027757462268932272 [Y1 Z2 Y3 Z7] +
0.00010502189355807878 [Y1 Z2 Z3 Y6 X7] +
0.09899076578197372 [Z1 Z2 Z3 Z4 Z5 Z7] +
-0.0010456709574065426 [X3 Y6 Y7] +
0.008434569756845402 [X4 X5] +
0.008434569756845402 [Y4 Y5] +
0.2707726623751814 [Z4 Z5] +
0.11409163501021498 [Z6 Z7]


First QWC group
-3.947119127137732 [] +
0.05362141072261472 [X

## NH<sub>3</sub>
Skip getting commuting groups as these steps are intractable. Skip straight ahead to getting QWC groups.

Warning: to save time you might want to skip applying the z-basis transformations with `apply_uzs=False`

In [19]:
mcalc = MeasurementGrouper('nh3', apply_uzs=True)
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	16 orbitals
2) Optionally apply tapering
	14 orbitals
More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups
	959 QWC groups, a 3.76 x reduction from 3609 terms
6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First QWC group
-34.413644501497416 [] +
-0.00081303365904289 [Y0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11 X12 X13] +
1.0772098183363803 [Z1] +
0.13600689568262758 [Z1 Z2] +
0.10606016163902365 [Z1 Z3] +
0.13600689568262755 [Z1 Z4] +
0.10606016163902378 [Z1 Z5] +
0.14836407229820797 [Z1 Z6] +
0.12295464996510772 [Z1 Z7] +
0.13150100122363995 [Z1 Z8] +
0.10855121194291724 [Z1 Z9] +
0.13610484417123336 [Z1 Z10] +
0.7684575395870031 [Z2] +
0.14032184811826043 [Z2 Z3] +
0.10748692518481216 [Z2 Z4] +
0.11843189949596164 [Z2 Z5] +
0.12437215825428041 [Z2 Z6] +
0.13177108541979066 [Z2 Z7] +
0.10788739359761279 [Z2 Z8] +
0.12231181049825243 [Z2 Z9] +
0.10428256107393341 [Z2 Z10] +
0.7684575395870034 [Z3] +
0.11843189949596164 [Z3 Z4] +
0.10748692518481216 [Z3 Z5] +
0.13177108541979066 [Z3 Z6] +
0.12437215825428041 [Z3 Z7] +
0.12231181049825243 [Z3 Z8] +
0.10788739359761279 [Z3 Z9] +
0.1375221381619795 [Z3 Z10] +
0.7684575395870028 [Z4] +
0.14032184811826073 [Z4 Z5] +
0.12437215825428051 [Z4 Z6] +

## N<sub>2</sub>
Skip getting commuting groups as these steps are intractable. Skip straight ahead to getting QWC groups.

Warning: to save time you might want to skip applying the z-basis transformations with `apply_uzs=False`

In [18]:
mcalc = MeasurementGrouper('n2', apply_uzs=True)
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	20 orbitals
2) Optionally apply tapering
	16 orbitals
More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups
	680 QWC groups, a 4.34 x reduction from 2951 terms
6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First QWC group
-66.82850881574544 [] +
9.563301651472461 [Z0] +
-8.745925056048934e-05 [Z0 X1 Y7 Z8 Z9 Z10 Z11 Z12 Y13] +
0.04758098481143379 [Z0 X1 X14 Z15] +
0.18023887084369627 [Z0 Z2] +
-0.005824718860252173 [Z0 Z2 Z3 Z4 Z5 Z6 X14] +
0.003212498244432343 [Z0 Z2 Z4 Z6 Y7 Z8 Z10 Z12 Y13 Z15] +
-0.16818202264181578 [Z0 Z2 Z4 Z6 Z8 Z10 Z12 Z15] +
0.16494698146964537 [Z0 Z3] +
0.1574850800510944 [Z0 Z4] +
0.16839241530030746 [Z0 Z5] +
0.1650909592962415 [Z0 Z6] +
0.1681931653702476 [Z0 Z8] +
0.1658266762913248 [Z0 Z9] +
0.17749543250218083 [Z0 Z10] +
0.17403023288530975 [Z0 Z11] +
0.17749543250218075 [Z0 Z12] +
0.20707041559534994 [Z0 Z15] +
0.0011148794780630508 [X1 Z2 Z3 Z4 Z5 Z6 Y7 Z8 Z9 Z10 Z11 Z12 Y13 X14] +
-0.0032834895171798494 [X1 Z2 Z3 Z4 Z5 Z6 Z15] +
1.7411017750776425 [Z2] +
0.12590654018141012 [Z2 Z3] +
0.10809968817868348 [Z2 Z4] +
0.13333369645075033 [Z2 Z5] +
0.11882440493025873 [Z2 Z6] +
0.148444898668402 [Z2 Z8] +
0.12379026212730872 [Z2 Z9] +
0.14445795674212672 [